#**Answer Generation 任務**
資料集引用台達閱讀理解資料集









### pytorch環境設定(https://pytorch.org)
選擇適合的版本並複製指令貼到本專案

In [0]:
!pip install torch torchvision

In [0]:
#確定有安裝成功
import torch
print(torch.__version__)
torch.cuda.is_available()

1.5.0+cu101


True

### **取得 google drive 存取權限**

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
cd /content/drive/My Drive/Colab Notebooks/Bert/AnswerGeneration/

/content/drive/My Drive/Colab Notebooks/Bert/AnswerGeneration


In [0]:
!ls

albert	       test.json      trained_model2  train.json
predict.ipynb  trained_model  train.ipynb


In [0]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### **下載dataset**

In [0]:
import os
def fileExists(filepath):
  return  os.path.exists(filepath)
#我把資料集放到我的github
if fileExists('train.json')!=True:
  !wget -O train.json https://raw.githubusercontent.com/harry83528/NLP-BERT-Experience/master/AnswerGeneration/DataSet/DRCD_training.json
if fileExists('test.json')!=True:
  !wget -O test.json https://raw.githubusercontent.com/harry83528/NLP-BERT-Experience/master/AnswerGeneration/DataSet/DRCD_test.json

In [0]:
!ls

albert	       test.json      trained_model2  train.json
predict.ipynb  trained_model  train.ipynb


##解析json

In [0]:
!pip install ijson

In [0]:
import ijson
# 讀取數據
def LoadData(filepath):
    context = []
    question = []
    answersText = []
    count=0
    contextId=''
    with open(filepath) as file:
        parser = ijson.parse(file)
        contextId=''
        context2=''
        question2=''
        for prefix, event, value in parser:
            # print('prefix',prefix)
            # print('event',event)
            # print('value',value)
            if prefix=="data.item.paragraphs.item.id":
              #print('contextId',value)          
              contextId=value
            if prefix=="data.item.paragraphs.item.context":
              #print('context',value)
              context2=value
            if prefix=="data.item.paragraphs.item.qas.item.question":
              question2=value
              #print('question',value)
              #print('contextId',contextId)              
            if prefix=="data.item.paragraphs.item.qas.item.answers.item.text":
              #print('context2',context2)
              #print('answersText',value)
              #print('contextId',contextId)
              context.append(context2)
              question.append(question2)
              answersText.append(value+"[SEP]")
            #print("---"*50)
            # count=count+1
            # if count==51:
            #   break;
    return context,question,answersText

##下載Albert並放在albert目錄

In [0]:
!git clone https://github.com/harry83528/albert-zh-for-pytorch-transformers.git albert

fatal: destination path 'albert' already exists and is not an empty directory.


### **安裝所需的函式庫-HuggingFace 團隊將 GitHub 專案大翻新並更名成 transformers(Install the Hugging Face Library)**
https://github.com/huggingface/transformers

In [0]:
!pip install transformers

In [0]:
%tensorflow_version 2.x

In [0]:
def use_model(model_name, config_file_path, model_file_path, vocab_file_path, num_labels):
    # 選擇模型並加載設定
    if(model_name == 'bert'):
        from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
        model_config, model_class, model_tokenizer = (BertConfig, BertForSequenceClassification, BertTokenizer)
        config = model_config.from_pretrained(config_file_path,num_labels = num_labels)
        model = model_class.from_pretrained(model_file_path, from_tf=bool('.ckpt' in 'bert-base-chinese'), config=config)
        tokenizer = model_tokenizer(vocab_file=vocab_file_path)
        return model, tokenizer
    elif(model_name == 'albert'):
        from albert.albert_zh import AlbertConfig, AlbertTokenizer, AlbertForMaskedLM
        model_config, model_class, model_tokenizer = (AlbertConfig, AlbertForMaskedLM, AlbertTokenizer)
        config = model_config.from_pretrained(config_file_path)
        #config = model_config.from_pretrained(config_file_path,num_labels = num_labels) # AlbertForMaskedLM 不需分類,不用設num_labels
        model = model_class.from_pretrained(model_file_path, config=config)
        tokenizer = model_tokenizer.from_pretrained(vocab_file_path)
        return model, tokenizer

### **指定想要用的模型是哪一種**
到以下網址查詢模型與函數資訊
https://huggingface.co/transformers

In [0]:
#通常我會在這設一些通用模型參數
# PRETRAINED_MODEL_Name ="bert-base-chinese"  # 指定 想要用的預訓練模型
trained_Model_Path = "trained_model3"

## **引入需要⽤到的函式庫**

In [0]:
import os
import pickle
import torch
from transformers import BertConfig, BertTokenizer, BertForMaskedLM, AdamW # 此任務不用
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F # 激勵函數
import time

### **載入資料並將資料轉換成輸入格式(Token Embeddings、Segment Embeddings、Position Embeddings)**

In [0]:
def to_feature(tokenizer, context, question, answer, token_embeddings, segement_embeddings, attention_mask, masked_labels, max_seq_len):
    segement_id = []
    attention_id = []
    masked_id = []

    context_string = "[CLS]" + context + "[SEP]"
    context_tokenize = tokenizer.tokenize(context_string)
    for i in range(len(context_tokenize)):
        segement_id.append(0) # 第0組
        attention_id.append(1) # attention都是1告訴bert是有意義的文字
        masked_id.append(-1)  # 不是被mask的字都是-1

    question_string = question + "[SEP]"
    question_tokenize = tokenizer.tokenize(question_string)
    for i in range(len(question_tokenize)):
        segement_id.append(1) # 第1組
        attention_id.append(1) # attention都是1告訴bert是有意義的文字
        masked_id.append(-1)  # 不是被mask的字都是-1

    answer_tokenize = tokenizer.tokenize(answer)
    #print('answer_tokenize:',answer_tokenize); #['西', '元', '1994', '年', '5', '月', '28', '日','<SEP>']
    for index, answer in enumerate(answer_tokenize):
        input_string = context_tokenize + question_tokenize
        token_id = []
        input_segement_id = segement_id + []
        input_attention_id = attention_id + []
        input_masked_id = masked_id + []
        
        for i in range(index):
            input_string.append(answer_tokenize[i])
            input_segement_id.append(2) # 第2組
            input_attention_id.append(1) # attention都是1告訴bert是有意義的文字
            input_masked_id.append(-1)  # 不是被mask的字都是-1

        input_string.append('[MASK]')
        input_segement_id.append(2) # 第2組
        input_attention_id.append(1) # attention都是1告訴bert是有意義的文字
        input_masked_id.append(tokenizer.convert_tokens_to_ids(answer))  # 被mask的掉字給該單詞tokens id

        token_id = tokenizer.convert_tokens_to_ids(input_string)
        # print('input_string',input_string)
        # print('token_id',token_id)
        # print('input_segement_id',input_segement_id)
        # print('input_attention_id',input_attention_id)
        # print('input_masked_id',input_masked_id)
        # print('---'*20)

        token_embeddings.append(token_id)
        segement_embeddings.append(input_segement_id)
        attention_mask.append(input_attention_id)
        masked_labels.append(input_masked_id)

        if len(token_id) > max_seq_len:
            max_seq_len = len(token_id)

    return max_seq_len
def convert_data_to_feature(filepath,tokenizer):

    context,question,answersText = LoadData(filepath)

    print(len(context),len(question),len(answersText))
    print(context[1000])
    print(question[1000])
    print(answersText[1000])

    token_embeddings = []
    segement_embeddings = []
    attention_mask = []
    masked_labels = []

    max_seq_len = 0
    conform_count = 0
    for i in range(0,len(context)-1):
        context_word_piece_list=tokenizer.tokenize(context[i])
        question_word_piece_list=tokenizer.tokenize(question[i])
        answersText_word_piece_list=tokenizer.tokenize(answersText[i])
        if len(context_word_piece_list) <= 450 and len(question_word_piece_list) <= 42 and len(answersText_word_piece_list) <= 16:
            max_seq_len = to_feature(tokenizer, context[i], question[i], answersText[i],token_embeddings,segement_embeddings,attention_mask,masked_labels,max_seq_len)
            conform_count =conform_count+1

    print("最大長度:",max_seq_len)
    print("有符合限制長度的總數",conform_count)

    assert max_seq_len <= 512 # 小於BERT-base長度限制
    max_seq_len = 512

    # 不足長度都補齊長度,都補0
    for item in token_embeddings:
        while len(item)<max_seq_len:
            item.append(0)

    for item in segement_embeddings:
        while len(item)<max_seq_len:
            item.append(0)

    for item in attention_mask:
        while len(item)<max_seq_len:
            item.append(0)

    for item in masked_labels:
        while len(item)<max_seq_len:
            item.append(-1)
    
    assert len(token_embeddings) == len(segement_embeddings) and len(token_embeddings) == len(attention_mask) and len(token_embeddings) == len(masked_labels)

    #為了使⽤⽅便，把三個input embedding及 label 包成 object======================
    data_features = {'token_embeddings':token_embeddings,
                    'segement_embeddings':segement_embeddings,
                    'attention_mask':attention_mask,
                    'masked_labels':masked_labels}

    return data_features
       

### **輸入格式轉Dataset**

In [0]:
def makeDataset(data_feature):
    token_embeddings = data_feature['token_embeddings']
    segement_embeddings = data_feature['segement_embeddings']
    attention_mask = data_feature['attention_mask']
    masked_labels = data_feature['masked_labels']

    all_token_embeddings = torch.tensor([token_id for token_id in token_embeddings], dtype=torch.long)
    all_segement_embeddings = torch.tensor([segement_id for segement_id in segement_embeddings], dtype=torch.long)
    all_attention_mask = torch.tensor([attention_id for attention_id in attention_mask], dtype=torch.long)
    all_masked_labels = torch.tensor([masked_lm_id for masked_lm_id in masked_labels], dtype=torch.long)

    return TensorDataset(all_token_embeddings, all_segement_embeddings, all_attention_mask, all_masked_labels)

## **確定 Embedding 是否正確**

## **分割數據**
把資料切成訓練集跟測試集
對於 Supervised Learning 來說，我們使⽤標註資料（Labeled Data）來做訓練
因此我們需要訓練資料（Training Data）
但是我們蒐集到的資料卻不能全部拿來做訓練
因為我們必須要保留⼀些當作測試資料（Testing Data）來評估模型表現
這些資料必須跟訓練資料是完全不同的，否則就有作弊的嫌疑
所以我們會把我們的資料切成訓練集與測試集，通常會保留比較多當作訓練資料
（60%~80%），其餘做測試資料

In [0]:
def split_dataset(full_dataset, split_rate=0.8):  
    train_size = int(split_rate * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    return train_dataset,test_dataset

### **Fine-Tuning**


In [0]:
# 計算正確值 (此次任務為生成不需要)
# def compute_accuracy(y_pred, y_target):
#     _, y_pred_indices = y_pred.max(dim=1)
#     n_correct = torch.eq(y_pred_indices, y_target).sum().item()
#     return n_correct / len(y_pred_indices) * 100

In [0]:
# RuntimeError: Error(s) in loading state_dict for AlbertForMaskedLM:
# 	size mismatch for bert.embeddings.token_type_embeddings.weight: copying a param with shape torch.Size([2, 312]) from checkpoint, the shape in current model is torch.Size([3, 312])
# if len(error_msgs) > 0:
#     raise RuntimeError('Error(s) in loading state_dict for {}:\n\t{}'.format(
#                         model.__class__.__name__, "\n\t".join(error_msgs)))
#設定模型參數 Start =============================================================
#Step1:初始化加載模型 Start======================================================
model_setting = {
    "model_name":"albert", 
    "config_file_path":"albert/albert_tiny/config.json", 
    "model_file_path":"albert/albert_tiny/pytorch_model.bin", 
    "vocab_file_path":"albert/albert_tiny/vocab.txt",
    "num_labels": -1 # 分幾類(AlbertForMaskedLM 不需分類,不用設num_labels,在這隨便亂設)
}
#初始化加載模型
model, tokenizer = use_model(**model_setting)
# config = BertConfig.from_pretrained(PRETRAINED_MODEL_Name)
# model = BertForMaskedLM.from_pretrained(PRETRAINED_MODEL_Name, from_tf=bool('.ckpt' in PRETRAINED_MODEL_Name), config=config)
#Step1:初始化加載模型 END========================================================

#Step2:指定硬體裝置 Start========================================================
# setting device
#你電腦的 GPU 能否被 PyTorch 調用,如果不行就使用CPU 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("using device",device)
model.to(device)
#Step2:指定硬體裝置 END==========================================================

#Step3:將訓練資料讀入並且組建BERT輸入格式 Start===================================
train_data_feature = convert_data_to_feature('train.json',tokenizer)
test_data_feature = convert_data_to_feature('test.json',tokenizer)
 #Step3:將訓練資料讀入並且組建BERT輸入格式 END====================================

#Step4:將組建好的輸入格式轉換成tensor格式，並且建立dataset與dataloader Srart======
# full_dataset = makeDataset(train_data_feature)
# train_dataset, test_dataset = split_dataset(full_dataset, 0.8)
train_dataset = makeDataset(train_data_feature)
test_dataset = makeDataset(test_data_feature)

train_dataloader = DataLoader(train_dataset ,batch_size=2 ,shuffle=True)
test_dataloader = DataLoader(test_dataset ,batch_size=1 ,shuffle=True)
#Step4:將組建好的輸入格式轉換成tensor格式，並且建立dataset與dataloader END========

#準備優化器 Start===============================================================
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
Learning_rate = 5e-6       # 學習率
optimizer = AdamW(optimizer_grouped_parameters, lr=Learning_rate, eps=1e-8)
#準備優化器 End=================================================================


#設定模型參數 End===============================================================

using device cuda
26936 26936 26936
滿洲國境內鐵路線稠密，鐵道運輸發達。鐵路由南滿洲鐵道株式會社經營，最主要的幹線為南滿鐵路。1935年，東清鐵路由滿洲國向蘇聯收購，再由滿鐵繼續鋪設鐵路。滿洲國以1.6億日元的價格收購長春至哈爾濱以及滿洲里至綏芬河的鐵路。其他重要鐵路還有丹東至奉天的安奉線、新京至圖們的京圖線、四平至齊齊哈爾的平齊線等。到1939年，路線全長已超過一萬公里，1945年達到11479公里，成為當時世界鐵道運輸最發達的國家之一。相較之下，1949年時全中國鐵路總里程僅22000公里。在大連與哈爾濱之間運營的「超特急」列車「亞細亞號」為當時滿鐵的象徵。1943年滿洲國公路總里程近6萬公里，而1949年，中國公路總里程僅有8.09萬公里。航運部分，主要港口有大連港和營口港；內河水運主要集中在松花江地區。
營運滿洲國境內的鐵路的是什麼公司？
南滿洲鐵道株式會社[SEP]
最大長度: 499
有符合限制長度的總數 17130
6986 6986 6986
從19世紀末至20世紀初湧入多倫多的移民主要是德國人、義大利人和來自東歐的猶太人。華人、俄羅斯人、波蘭人和其他東歐國家的移民後來相繼抵達。縱使人口增長迅速，直至1920年代，多倫多的人口和經濟影響力還是不及歷史較悠久的蒙特婁，但多倫多證券交易所卻於1934年成為全國最大的交易所。二戰後，來自東歐的難民，以及華人、義大利人和葡萄牙人陸續抵達。加拿大政府於1960年代末取消含有種族歧視成份的移民法後，更多移民從世界各個角落抵達多倫多。多倫多的人口於1951年越過一百萬大關，更於1971年增至二百萬人。而很多全國和跨國企業於1980年代有見於魁北克政局不穩，紛紛把總部從蒙特婁遷至多倫多和加拿大西部其他城市，也造就多倫多於1980年代取代蒙特婁成為全國最大城市和金融中心。
什麼時間之前加拿大最大的城市和金融中心並非多倫多？
1980年代[SEP]
最大長度: 504
有符合限制長度的總數 4308


### **開始訓練**

In [0]:
#Step5:建構training loop、開始訓練 Start=========================================
for epoch in range(5):
    
    All_train_correct = 0.0
    AllTrainLoss = 0.0
    count = 0
    model.zero_grad() #梯度歸零
    for batch_dict in train_dataloader:

        # 訓練模式
        model.train()
        
        batch_dict = tuple(t.to(device) for t in batch_dict)
        #print(batch_dict[0].shape) #torch.Size([4, 512])
        #print(batch_dict[1].shape) #torch.Size([4, 512])
        #print(batch_dict[2].shape) #torch.Size([4, 512])
        #print(batch_dict[3].shape) #torch.Size([4, 512])
        #順序對應產⽣Dataset那裡的dataset = TensorDataset(all_token_embeddings, all_segement_embeddings, all_attention_mask, all_masked_labels)
        outputs = model(
            input_ids = batch_dict[0],
            token_type_ids = batch_dict[1],
            attention_mask = batch_dict[2],
            masked_lm_labels = batch_dict[3]
            )
        loss, logits = outputs[:2]   

        AllTrainLoss += loss.item()
        count += 1

        model.zero_grad()
        loss.backward()
        optimizer.step()
            
    Average_train_loss = round(AllTrainLoss/count, 3)

    # 測試模式
    model.eval()
    All_test_correct = 0.0
    AllTestLoss = 0.0
    count = 0
    for batch_dict in test_dataloader:
        batch_dict = tuple(t.to(device) for t in batch_dict)  #每⼀個Batch把資料從CPU RAM 推到 GPU RAM

        outputs = model(
            input_ids = batch_dict[0],
            token_type_ids = batch_dict[1],
            attention_mask = batch_dict[2],
            masked_lm_labels = batch_dict[3]
            )
        loss, logits = outputs[:2]

        AllTestLoss += loss.item()

        count += 1
        
    Average_test_loss = round(AllTestLoss/count, 3)

    print('第' + str(epoch+1) + '次' + '訓練模式，loss為:' + str(Average_train_loss) + '，測試模式，loss為:' + str(Average_test_loss) )
    print ("時間 :", time.asctime( time.localtime(time.time())) )
    print('---'*20)

#Step5:建構training loop、開始訓練 END===========================================

第1次訓練模式，loss為:6.751，測試模式，loss為:6.081
時間 : Sun Jun 14 06:23:38 2020
------------------------------------------------------------
第2次訓練模式，loss為:5.92，測試模式，loss為:5.913
時間 : Sun Jun 14 06:43:38 2020
------------------------------------------------------------
第3次訓練模式，loss為:5.737，測試模式，loss為:5.771
時間 : Sun Jun 14 07:03:38 2020
------------------------------------------------------------
第4次訓練模式，loss為:5.507，測試模式，loss為:5.495
時間 : Sun Jun 14 07:23:47 2020
------------------------------------------------------------
第5次訓練模式，loss為:5.099，測試模式，loss為:4.999
時間 : Sun Jun 14 07:43:48 2020
------------------------------------------------------------


### **儲存模型**

In [0]:
#創⼀個資料夾存模型
if not os.path.isdir(trained_Model_Path):
    os.mkdir(trained_Model_Path)
#儲存
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(trained_Model_Path)

In [0]:
print('train end')

train end
